In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import json
import time
import datetime
from collections import deque

# Load emotion recognition model and metadata
model = tf.keras.models.load_model('emotion_recognition_5class_final.h5')
with open('emotion_recognition_5class_final_metadata.json', 'r') as f:
    metadata = json.load(f)

# Load DNN face detector
face_net = cv2.dnn.readNetFromCaffe(
    'deploy.prototxt',
    'res10_300x300_ssd_iter_140000.caffemodel'
)

# Preprocessing function
def preprocess_image(image):
    image = cv2.resize(image, (48, 48))
    if len(image.shape) == 3 and image.shape[2] > 1:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = image / 255.0
    image = image.reshape(1, 48, 48, 1)
    return image

# Emotion colors
emotion_colors = {
    "Engaged": (46, 250, 87),     # Dark green
    "Frustrated": (51, 51, 204),  # Red (BGR format)
    "Disengaged": (204, 51, 51),  # Blue (BGR format)
    "Surprise": (0, 215, 255),    # Yellow (BGR format)
    "Neutral": (96, 96, 96)       # Gray
}

emotion_counts = {emotion: 0 for emotion in emotion_colors}
emotion_timestamps = {emotion: [] for emotion in emotion_colors}
start_time = time.time()
prediction_buffer = deque(maxlen=10)

cap = cv2.VideoCapture(1)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # DNN face detection
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300),
                                 (104.0, 177.0, 123.0))
    face_net.setInput(blob)
    detections = face_net.forward()

    h, w = frame.shape[:2]
    faces = []

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.6:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            x1, y1, x2, y2 = box.astype("int")
            faces.append((x1, y1, x2 - x1, y2 - y1))

    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]
        processed_img = preprocess_image(face_img)

        raw_preds = model.predict(processed_img)[0]
        prediction_buffer.append(raw_preds)
        avg_preds = np.mean(prediction_buffer, axis=0)

        pred_class = np.argmax(avg_preds)
        emotion = metadata['emotion_map'][str(pred_class)]
        confidence = float(avg_preds[pred_class])

        current_time = datetime.datetime.now()
        emotion_counts[emotion] += 1
        emotion_timestamps[emotion].append(current_time)

        box_color = emotion_colors.get(emotion, (255, 255, 255))
        cv2.rectangle(frame, (x, y), (x+w, y+h), box_color, 2)

        label = f'{emotion}'
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX,
                    0.9, box_color, 2, lineType=cv2.LINE_AA)

        bar_x = x + w + 10
        bar_y = y
        bar_width = 50
        bar_height = h // len(metadata['emotion_map'])

        for i, (key, name) in enumerate(metadata['emotion_map'].items()):
            prob = avg_preds[int(key)]
            bar_length = int(prob * bar_width)
            color = emotion_colors.get(name, (255, 255, 255))
            y_pos = bar_y + i * bar_height

            cv2.rectangle(frame, (bar_x, y_pos),
                (bar_x + bar_width, y_pos + bar_height - 4),
                (0, 0, 0), -1, lineType=cv2.LINE_AA)
            cv2.rectangle(frame, (bar_x, y_pos),
                (bar_x + bar_width, y_pos + bar_height - 4),
                color, 1, lineType=cv2.LINE_AA)
            cv2.rectangle(frame, (bar_x, y_pos),
                (bar_x + bar_length, y_pos + bar_height - 4),
                color, -1, lineType=cv2.LINE_AA)
            cv2.putText(frame, name, (bar_x + bar_width + 6, y_pos + bar_height - 6),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.42, (50, 50, 50), 2, lineType=cv2.LINE_AA)
            cv2.putText(frame, name, (bar_x + bar_width + 6, y_pos + bar_height - 6),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.42, color, 1, lineType=cv2.LINE_AA)

    session_time = time.time() - start_time
    stats_y = 30
    cv2.putText(frame, f"Session time: {int(session_time)}s", (10, stats_y), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, lineType=cv2.LINE_AA)

    for idx, (emotion_name, count) in enumerate(emotion_counts.items()):
        stats_y += 25
        color = emotion_colors.get(emotion_name, (255, 255, 255))
        cv2.putText(frame, f"{emotion_name}: {count}", (10, stats_y), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 1, lineType=cv2.LINE_AA)

    cv2.imshow('Emotion Recognition', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Save session data
data_to_save = {
    "total_session_time": time.time() - start_time,
    "emotion_counts": emotion_counts,
    "timestamp": datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
}

with open(f"emotion_session_data_{data_to_save['timestamp']}.json", 'w') as f:
    json.dump(data_to_save, f, indent=4, default=str)

print("Emotion data saved to file.")
cap.release()
cv2.destroyWindow('Emotion Recognition')
cv2.waitKey(1)
time.sleep(0.5)

# generate a report based on the saved data with graphs
def generate_report(data_file):
    import matplotlib.pyplot as plt
    from matplotlib.colors import rgb2hex
    
    with open(data_file, 'r') as f:
        data = json.load(f)
    
    # Text report portion
    report = f"Emotion Recognition Session Report\n"
    report += f"Session Time: {data['total_session_time']:.2f} seconds\n"
    report += f"Timestamp: {data['timestamp']}\n\n"
    report += "Emotion Counts:\n"
    
    total_emotions = sum(data['emotion_counts'].values())
    
    for emotion, count in data['emotion_counts'].items():
        percentage = (count / total_emotions * 100) if total_emotions > 0 else 0
        report += f"{emotion}: {count} ({percentage:.1f}%)\n"
    
    # Create visualizations
    plt.figure(figsize=(15, 10))
    
    # Pie chart
    plt.subplot(1, 2, 1)
    labels = list(data['emotion_counts'].keys())
    counts = list(data['emotion_counts'].values())
    
    # Map emotion_colors to pie chart colors
    colors = [rgb2hex([c[2]/255, c[1]/255, c[0]/255]) for c in [emotion_colors.get(emotion, (255, 255, 255)) for emotion in labels]]
    
    # Only include emotions with count > 0 in the pie chart
    non_zero_labels = []
    non_zero_counts = []
    non_zero_colors = []
    
    for i, count in enumerate(counts):
        if count > 0:
            non_zero_labels.append(labels[i])
            non_zero_counts.append(count)
            non_zero_colors.append(colors[i])
    
    if sum(non_zero_counts) > 0:  # Only create pie chart if there's data
        plt.pie(non_zero_counts, labels=non_zero_labels, colors=non_zero_colors, 
                autopct='%1.1f%%', shadow=True, startangle=90)
        plt.axis('equal')
        plt.title('Emotion Distribution')
    
    # Bar chart
    plt.subplot(1, 2, 2)
    plt.bar(labels, counts, color=colors)
    plt.title('Emotion Counts')
    plt.xlabel('Emotion')
    plt.ylabel('Count')
    for i, v in enumerate(counts):
        plt.text(i, v + 0.5, str(v), ha='center')
    
    # Save the figure
    chart_file = f"emotion_session_chart_{data['timestamp']}.png"
    plt.tight_layout()
    plt.savefig(chart_file)
    plt.close()
    
    # Add chart file info to report
    report += f"\nEmotional analysis charts saved to: {chart_file}\n"
    
    # Create a summary section
    report += "\nSUMMARY ANALYSIS:\n"
    if total_emotions > 0:
        dominant_emotion = max(data['emotion_counts'].items(), key=lambda x: x[1])[0]
        report += f"- Dominant emotion: {dominant_emotion}\n"
        
        # Calculate engagement score (engaged vs disengaged ratio)
        engaged = data['emotion_counts'].get('Engaged', 0)
        disengaged = data['emotion_counts'].get('Disengaged', 0)
        
        if (engaged + disengaged) > 0:
            engagement_ratio = engaged / (engaged + disengaged) if (engaged + disengaged) > 0 else 0
            report += f"- Engagement score: {engagement_ratio:.2f} ({engagement_ratio*100:.1f}%)\n"
            
            if engagement_ratio > 0.8:
                report += "  Interpretation: High engagement level\n"
            elif engagement_ratio > 0.5:
                report += "  Interpretation: Moderate engagement level\n"
            else:
                report += "  Interpretation: Low engagement level\n"
    else:
        report += "- No emotions detected during this session\n"
    
    return report, chart_file

# Generate and print the report
report, chart_file = generate_report(f"emotion_session_data_{data_to_save['timestamp']}.json")
print("\nGenerated Report:")
print(report)

# Save the report to a text file
report_file = f"emotion_session_report_{data_to_save['timestamp']}.txt"
with open(report_file, 'w') as f:
    f.write(report)
print(f"Report saved to {report_file}.")
print(f"Charts saved to {chart_file}.")

# Display the chart in the notebook
from IPython.display import Image, display
display(Image(filename=chart_file))

